In [282]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
from prophet import Prophet
import matplotlib.pyplot as plt
import matplotlib.style as style
import plotly.graph_objects as go
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
%matplotlib inline

In [283]:
df = pd.read_csv("C:/Users/ceitf/Angular/Flask/file.csv", encoding='Latin-1')
pd.DataFrame(df.dtypes, columns=['Type'])

,Type
Row ID,int64
Order ID,object
Order Date,object
Ship Date,object
Ship Mode,object
Customer ID,object
Customer Name,object
Segment,object
Country,object
City,object


In [284]:
df1 = df.dropna()
# df1.isnull().sum()

In [285]:
sale = df1.groupby('Order Date')['Sales'].sum().reset_index()


In [286]:
data = sale
data.rename(columns={'Order Date':'ds','Sales':'y'}, inplace = True)


In [287]:
data['ds'] = pd.to_datetime(data['ds'])
data = data.sort_values('ds')
print(data['ds'])

55    2014-01-03
66    2014-01-04
69    2014-01-05
72    2014-01-06
75    2014-01-07
         ...    
379   2017-12-26
383   2017-12-27
386   2017-12-28
390   2017-12-29
398   2017-12-30
Name: ds, Length: 1237, dtype: datetime64[ns]


In [288]:
data=data[(data['ds']>='2014-01-03') & (data['ds']<='2017-12-30')]

model = Prophet()
model.fit(data)
future_dates = pd.date_range(start='2018-01-01', end='2018-12-31', freq='D')
future = pd.DataFrame({'ds': future_dates})

forecast = model.predict(future)

fig = go.Figure(go.Bar(x=forecast['yhat'], y=forecast['ds']))
plt.show()

23:27:33 - cmdstanpy - INFO - Chain [1] start processing
23:27:33 - cmdstanpy - INFO - Chain [1] done processing


In [289]:
forecast['error'] = np.abs(data['y'] - forecast['yhat'])


fig = go.Figure(go.Scatter(x=forecast['ds'], y=forecast['error'], mode='lines',marker=dict(color='red'),
                         name='Error'))
fig.update_layout(title='Absolute Errors between Actual and Predicted Values', xaxis_title='Date', yaxis_title='Absolute Error')

all_dates = pd.date_range(start='2014-01-03', end='2018-12-31', freq='D')
disp = pd.DataFrame({'ds': all_dates})
disp = pd.merge(disp, data[['ds', 'y']], on='ds', how='left')
disp = pd.merge(disp, forecast[['ds', 'yhat']], on='ds', how='left')
disp['yhat'] = disp['yhat'].round(2)
disp.fillna(0, inplace=True)
disp['sum'] = disp['y'] + disp['yhat']

fig = go.Figure()
fig.add_trace(go.Bar(x=disp['ds'], y=disp['sum'], marker={'color': disp['sum'],'colorscale': 'Viridis'}))

# add a marker at the last data point
last_date = disp['ds']
last_value = disp['sum']
fig.add_trace(go.Scatter(x=[last_date], mode='markers+text',
                           marker_color='red',
                          textposition='top right'))

# customize the layout
fig.update_layout(title='Sum of Sales', xaxis_title='Date', yaxis_title='Sum')
fig.show()

In [290]:
disp.to_csv('predicted.csv', index = False)

             ds         y     yhat       sum
0    2014-01-03    16.448     0.00    16.448
1    2014-01-04   288.060     0.00   288.060
2    2014-01-05    19.536     0.00    19.536
3    2014-01-06  4407.100     0.00  4407.100
4    2014-01-07    87.158     0.00    87.158
...         ...       ...      ...       ...
1819 2018-12-27     0.000  3252.38  3252.380
1820 2018-12-28     0.000  3414.84  3414.840
1821 2018-12-29     0.000  2911.77  2911.770
1822 2018-12-30     0.000  2986.40  2986.400
1823 2018-12-31     0.000  3085.92  3085.920

[1824 rows x 4 columns]
